![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación de librerías
%matplotlib inline
import pandas as pd

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,False,False,False,True,False,False,False
11,13995,2014,39972,False,False,False,False,True,False,False
167,17941,2016,18989,False,False,False,False,False,True,False
225,12493,2014,51330,False,False,False,True,False,False,False
270,7994,2007,116065,False,True,False,False,False,False,False


In [3]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [4]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [5]:
# Celda 1
import numpy as np

max_depth = None
num_pct = 10
max_features = None
min_gain=0.001

def best_split(X, y, num_pct=10):
    features = range(X.shape[1])
    best_split = [0, 0, -np.inf]  
    
    X = X.copy()
    for col in X.select_dtypes(include=['bool']).columns:
        X[col] = X[col].astype(int)
    

    def mse(y):
        if len(y) == 0:
            return 0
        return np.mean((y - np.mean(y))**2)
    
    for j in features:
        column = X.iloc[:, j]
        
        if len(np.unique(column)) == 1:
            continue

        splits = np.percentile(column, np.linspace(0, 100, num_pct + 2)[1:-1]) 
        splits = np.unique(splits)
        
        for split in splits:
            mask = column < split
            y_left, y_right = y[mask], y[~mask]
            
            if len(y_left) == 0 or len(y_right) == 0:
                continue  

            mse_parent = mse(y)
            mse_left = mse(y_left)
            mse_right = mse(y_right)
            n_total = len(y)
            gain = mse_parent - (len(y_left)/n_total * mse_left + len(y_right)/n_total * mse_right)
            
            if gain > best_split[2]:
                best_split = [j, split, gain]
    
    return best_split

j, split, gain = best_split(X_train, y_train, 5)
j, split, gain

(0, 2014.0, 8706079.2690802)

🟢 Procedimiento y explicación

1. Ahora verifico el split seleccionado por la función para ver como es estan dividiendo los datos
2. Verifico que no este generando ramas vacias al momento de hacer la divisón, lo cual no sucede ya que al sumar el lado de la derecha e izquieda, da la cantidad usada en y_train
3. Ahora corro la función tree_grow con X_train y X_test

In [6]:
filter_l = X_train.iloc[:, j] < split

y_l = y_train.loc[filter_l]
y_r = y_train.loc[~filter_l]
y_train.shape[0], y_l.shape[0], y_r.shape[0]

(7031, 2270, 4761)

In [7]:
def tree_grow(X, y, level=0, min_gain=0.001, max_depth=None, num_pct=10):
    if len(y) <= 1:
        return {'value': np.mean(y), 'level': level, 'split': -1, 'n_samples': len(y)}
    
    j, split, gain = best_split(X, y, num_pct) 
    
    if gain < min_gain or (max_depth is not None and level >= max_depth):
        return {'value': np.mean(y), 'level': level, 'split': -1, 'n_samples': len(y)}
    
    mask = X.iloc[:, j] < split
    return {'split': [j, split],
        'value': np.mean(y),
        'level': level,
        'n_samples': len(y),
        'gain': gain,
        'left': tree_grow(X[mask], y[mask], level+1, min_gain, max_depth, num_pct),
        'right': tree_grow(X[~mask], y[~mask], level+1, min_gain, max_depth, num_pct)}

tree = tree_grow(X_train, y_train, max_depth=2, num_pct=10)
tree

{'split': [0, 2014.0],
 'value': 14508.073389276064,
 'level': 0,
 'n_samples': 7031,
 'gain': 8706079.2690802,
 'left': {'split': [0, 2012.0],
  'value': 10234.929955947136,
  'level': 1,
  'n_samples': 2270,
  'gain': 4019659.1499605672,
  'left': {'value': 8737.28071379547,
   'level': 2,
   'split': -1,
   'n_samples': 1457},
  'right': {'value': 12918.90897908979,
   'level': 2,
   'split': -1,
   'n_samples': 813}},
 'right': {'split': [0, 2017.0],
  'value': 16545.467968914094,
  'level': 1,
  'n_samples': 4761,
  'gain': 1524388.1958712116,
  'left': {'value': 15976.070045848192,
   'level': 2,
   'split': -1,
   'n_samples': 3926},
  'right': {'value': 19222.66107784431,
   'level': 2,
   'split': -1,
   'n_samples': 835}}}

🟢 Procedimiento y explicación

1. ahora con la función tree_predict, realizo predicciones usando X_train y luego nuevamente pero con X_test.
2. Genero las metricas del MAE y el RMSE para que el desempeño del test

In [8]:
def tree_predict(X, tree, proba=False):
    predicted = np.zeros(X.shape[0])
    
    if 'split' not in tree or tree['split'] == -1:
        return np.full(X.shape[0], tree['value'])
    
    j, split = tree['split']
    filter_l = X.iloc[:, j] < split
    X_l, X_r = X[filter_l], X[~filter_l]

    if 'left' in tree and 'right' in tree:
        predicted[filter_l] = tree_predict(X_l, tree['left'], proba)
        predicted[~filter_l] = tree_predict(X_r, tree['right'], proba)
    else: 
        predicted[:] = tree['value']
    
    return predicted

y_pred = tree_predict(X_train, tree)

df_predicciones = X_train.copy()
df_predicciones["Predicción"] = y_pred
df_predicciones["Real"] = y_train

df_predicciones

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,Predicción,Real
318288,2014,39988,False,False,False,False,False,True,False,15976.070046,18495
333638,2007,38904,False,False,False,False,True,False,False,8737.280714,10891
234716,2016,29607,False,False,False,False,False,True,False,15976.070046,18994
208752,2014,50756,False,False,False,False,False,True,False,15976.070046,13691
348892,2017,10506,False,False,False,False,False,True,False,19222.661078,19999
...,...,...,...,...,...,...,...,...,...,...,...
218085,2010,84057,False,False,False,False,True,False,False,8737.280714,9995
198455,2012,93770,False,True,False,False,False,False,False,12918.908979,12898
205572,2010,88644,False,False,True,False,False,False,False,8737.280714,8988
33541,2015,26180,False,False,False,False,False,True,False,15976.070046,18995


In [9]:
y_pred = tree_predict(X_test, tree)

df_predicciones = X_test.copy()
df_predicciones["Predicción"] = y_pred
df_predicciones["Real"] = y_test

df_predicciones

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,Predicción,Real
257343,2012,62048,False,False,False,False,True,False,False,12918.908979,9900
326011,2005,111565,False,True,False,False,False,False,False,8737.280714,6987
242354,2014,24203,False,False,False,False,False,True,False,15976.070046,15814
266376,2015,30475,False,False,False,False,False,False,True,15976.070046,17997
396954,2014,30498,False,False,False,False,False,False,True,15976.070046,18938
...,...,...,...,...,...,...,...,...,...,...,...
144298,2016,39544,False,False,False,False,False,True,False,15976.070046,17988
364521,2015,37109,False,False,False,False,False,False,True,15976.070046,19995
120072,2017,27341,False,False,False,False,False,True,False,19222.661078,18330
99878,2012,78959,False,True,False,False,False,False,False,12918.908979,10495


In [10]:
mae = np.mean(np.abs(y_test - y_pred))
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
print('El MAE del conjunto de test es:', mae)
print('El RMSE del conjunto de test es:', rmse)

El MAE del conjunto de test es: 1637.1349204706257
El RMSE del conjunto de test es: 2128.8874044922427


🟢 Interpretación de los resultados.

1. Teniendo en cuenta que el promedio de venta de los carros es de 14.538 dolares, según el MAE, el modelo se equivoca en promedio unos 1.637 dolares, lo que significaria un 11.3% del precio de los carros.
2. Para el caso de RMSE, que penaliza los errores mas grandes, el error ronda el 14.6%, lo que puede indicar que hay errores mucho mas grandes en las predicciones pero no lo suficiente para desestabilziar el modelo

### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de regresión y comenten sobre el desempeño del modelo.

In [11]:
import numpy as np

In [12]:
# Celda 2

# Bagging manual

# Paso 1: hacer muestras con bootstrap para cada arbol, del mismo tamaño de observaciones que en train
observaciones_por_muestra = X_train.shape[0]
cantidad_de_muestras = 10 # 10 arboles

# list comprehension para crear las 10 muestras
np.random.seed(123)
muestras = [np.random.choice(a=observaciones_por_muestra, size=observaciones_por_muestra, replace=True) for muestra in range(cantidad_de_muestras)]

for i in range(len(muestras)):
    print(f"Muestra {i} de tamaño {len(muestras[i])}: ")
    print(muestras[i])

Muestra 0 de tamaño 7031: 
[3582 3454 1346 ...  826  801 5657]
Muestra 1 de tamaño 7031: 
[6962 3408 2553 ... 6611  877 6412]
Muestra 2 de tamaño 7031: 
[1917 3131  384 ... 2876 6449 6557]
Muestra 3 de tamaño 7031: 
[3849 4565 6820 ... 6835 4643  639]
Muestra 4 de tamaño 7031: 
[2468 3608 1367 ... 3108 2961 4357]
Muestra 5 de tamaño 7031: 
[3142 1537 5966 ... 3224 6922 3396]
Muestra 6 de tamaño 7031: 
[6588 3753 1786 ... 2131 4627 5672]
Muestra 7 de tamaño 7031: 
[1289 3776  981 ... 1891 6034 6165]
Muestra 8 de tamaño 7031: 
[6079 3819 6976 ... 6478 2225 5072]
Muestra 9 de tamaño 7031: 
[3952 3276  896 ... 4589 1964  893]


In [13]:
# Paso 2 crear los arboles con las diferentes muestras creadas en el paso anterior
from sklearn.tree import DecisionTreeRegressor

# definicion del objeto de decision tree reg
treereg = DecisionTreeRegressor(max_depth=None, random_state=123)

# dataframe para guardar las predicciones (columnas = cantidad de arboles, filas = los indices de las observaciones en test)
y_pred_bag_manual = pd.DataFrame(index=X_test.index, columns=list(range(cantidad_de_muestras)))

y_pred_bag_manual.head()

,0,1,2,3,4,5,6,7,8,9
257343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
for i, sample in enumerate(muestras):
    # seleccionar por posicion (iloc) las observaciones en X_train
    xtrain = X_train.iloc[sample, :]
    ytrain = y_train.iloc[sample]
    treereg.fit(xtrain, ytrain)
    y_pred_bag_manual.iloc[:,i] = treereg.predict(X_test)

y_pred_bag_manual

,0,1,2,3,4,5,6,7,8,9
257343,14980.0,13993.0,13649.0,13649.0,11788.0,13649.0,13993.0,13990.0,13990.0,13993.0
326011,6492.0,5995.0,5995.0,6987.0,5995.0,5995.0,5995.0,6987.0,5995.0,5995.0
242354,16491.0,16995.0,16491.0,15997.0,15997.0,16491.0,17591.0,16995.0,17404.0,16491.0
266376,21990.0,21990.0,22500.0,21990.0,15900.0,21990.0,22500.0,21990.0,15813.0,21990.0
396954,15988.0,16951.0,15988.0,15988.0,15988.0,17900.0,16951.0,16951.0,15988.0,15988.0
...,...,...,...,...,...,...,...,...,...,...
144298,13836.0,14800.0,14800.0,14800.0,14800.0,14681.0,14800.0,14800.0,13836.0,13836.0
364521,15999.0,14995.0,15999.0,16900.0,15999.0,15999.0,17300.0,15999.0,16900.0,15000.0
120072,23533.0,23533.0,20000.0,17700.0,17700.0,23533.0,17700.0,23533.0,23533.0,20000.0
99878,12989.0,12995.0,12989.0,12995.0,12991.0,12991.0,10995.0,12991.0,12991.0,12893.0


In [15]:
# desempeño de cada arbol
from sklearn.metrics import mean_squared_error

errores_bag_manual = []
for i in range(cantidad_de_muestras):
    error = np.sqrt(mean_squared_error(y_pred_bag_manual.iloc[:,i], y_test))
    errores_bag_manual.append(error)
    print(f"Arbol {i} tiene un error: {error}")

Arbol 0 tiene un error: 2141.613353645869
Arbol 1 tiene un error: 2136.3519863123465
Arbol 2 tiene un error: 2122.718759132052
Arbol 3 tiene un error: 2087.278992468617
Arbol 4 tiene un error: 2168.518742842026
Arbol 5 tiene un error: 2113.8811455834793
Arbol 6 tiene un error: 2127.933470769012
Arbol 7 tiene un error: 2184.414847251443
Arbol 8 tiene un error: 2138.1071697514985
Arbol 9 tiene un error: 2132.097520859104


In [16]:
# Paso 3: agregar las predicciones de los arboles 

# agregar una columna que promedie las demas columnas para obtener la prediccion del bagging
y_pred_bag_manual['final'] = y_pred_bag_manual.mean(axis=1)
y_pred_bag_manual.head()

,0,1,2,3,4,5,6,7,8,9,final
257343,14980.0,13993.0,13649.0,13649.0,11788.0,13649.0,13993.0,13990.0,13990.0,13993.0,13767.4
326011,6492.0,5995.0,5995.0,6987.0,5995.0,5995.0,5995.0,6987.0,5995.0,5995.0,6243.1
242354,16491.0,16995.0,16491.0,15997.0,15997.0,16491.0,17591.0,16995.0,17404.0,16491.0,16694.3
266376,21990.0,21990.0,22500.0,21990.0,15900.0,21990.0,22500.0,21990.0,15813.0,21990.0,20865.3
396954,15988.0,16951.0,15988.0,15988.0,15988.0,17900.0,16951.0,16951.0,15988.0,15988.0,16468.1


In [17]:
# medir el desempeño
error_bag_manual = np.sqrt(mean_squared_error(y_pred_bag_manual['final'],y_test))

errores_bag_manual.append(error_bag_manual)

print(f"Error al promediar todos los arboles {error_bag_manual} ")

Error al promediar todos los arboles 1796.4355868399332 


In [18]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1)

ax.bar(x=y_pred_bag_manual.columns.astype(str), height=errores_bag_manual) 
ax.set_title("Errores de Arboles + Final = Bagging")

Text(0.5, 1.0, 'Errores de Arboles + Final = Bagging')

### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de regresión y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [19]:
# Celda 3
from sklearn.ensemble import BaggingRegressor

n_features = X_train.shape[1]
max_features = int(np.log(n_features))  # logaritmo natural

bagreg = BaggingRegressor(DecisionTreeRegressor(max_features=max_features), n_estimators=10, bootstrap=True, oob_score=True, random_state=1)
bagreg.fit(X_train, y_train)
y_pred_bag_libreria = bagreg.predict(X_test)
y_pred_bag_libreria

array([13311.4,  6774.8, 16692.6, ..., 22596.4, 12990.6, 11965. ])

In [20]:
# medir el desempeño
error_bag_libreria = np.sqrt(mean_squared_error(y_pred_bag_libreria, y_test))
print(f"El error de Bagging con libreria sklearn es {error_bag_libreria}")

El error de Bagging con libreria sklearn es 1824.292388455352


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para regresión  y comenten sobre el desempeño del modelo.

In [21]:
# Celda 4


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [22]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de regresión con la librería sklearn y comenten sobre el desempeño del modelo.

In [23]:
# Celda 6


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [24]:
# Celda 7


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [25]:
# Celda 8
